# GPU Device Detection and Sentiment Analysis

Notebook ini mendemonstrasikan cara mendeteksi perangkat GPU yang tersedia (CUDA/MPS/CPU) dan menggunakan pipeline sentiment analysis dari Hugging Face Transformers.

In [1]:
import torch
use_cuda = torch.cuda.is_available()

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:', torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',
          torch.cuda.get_device_properties(0).total_memory/1e9)

In [ ]:
# Additional GPU details using GPUtil
try:
    import GPUtil

    gpus = GPUtil.getGPUs()

    for gpu in gpus:
        print("GPU ID:", gpu.id)
        print("GPU Name:", gpu.name)
        print("GPU Utilization:", gpu.load * 100, "%")
        print("GPU Memory Utilization:", gpu.memoryUtil * 100, "%")
        print("GPU Temperature:", gpu.temperature, "C")
        print("GPU Total Memory:", gpu.memoryTotal, "MB")
except ImportError:
    print("GPUtil not installed. Install with: pip install gputil")

In [2]:
def get_device():
    if torch.cuda.is_available(): # Check for CUDA sama kaya cuda:0
        device = "cuda"
    elif torch.backends.mps.is_available(): # Check for MPS sama kaya mps:0
        device = "mps"
    else:
        device = "cpu"
    print(f"✅ Using device: {device}")
    return device

In [3]:
from transformers import pipeline

device = get_device()

classifier = pipeline(
    "text-classification",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=device
)

/Users/zeihanaulia/Programming/research/agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Using device: mps


Device set to use mps


## Penjelasan Device Setting

Apa bedanya kalau di sini device di set dan tidak?

- Jika device di set, maka model akan dijalankan di perangkat yang ditentukan (CPU, CUDA, atau MPS).
- Jika tidak di set, maka model akan menggunakan perangkat default (biasanya CPU).
- Hugging Face (via transformers.pipeline) mendeteksi otomatis bahwa device mps:0 tersedia dan memilihnya sendiri.
- Tapi ini bergantung pada versi PyTorch dan Transformers — di beberapa versi lama, kalau lo gak set device, dia default ke CPU, bukan MPS.

Maksud dari :0 itu apa ya?
- :0 menunjukkan bahwa kita menggunakan GPU pertama (indeks dimulai dari 0). 
- Jika ada beberapa GPU, kita bisa memilih GPU mana yang ingin digunakan.

In [4]:
result = classifier("I love Hugging Face!")
print(result)

[{'label': 'POSITIVE', 'score': 0.9998641014099121}]


## MPS vs CUDA

MPS (Metal Performance Shaders) adalah framework akselerasi hardware yang dikembangkan oleh Apple untuk perangkat macOS dan iOS yang menggunakan GPU berbasis arsitektur Metal.

CUDA (Compute Unified Device Architecture) adalah platform komputasi paralel dan model pemrograman yang dikembangkan oleh NVIDIA untuk GPU mereka.

Keduanya memungkinkan pemrosesan paralel yang efisien, tetapi MPS dirancang khusus untuk perangkat Apple, sedangkan CUDA dirancang untuk GPU NVIDIA.

Secara performa, CUDA umumnya lebih matang dan memiliki dukungan yang lebih luas dalam komunitas pembelajaran mesin dibandingkan MPS, karena telah ada lebih lama dan digunakan secara luas di berbagai aplikasi.

Perbedaan utama antara MPS dan CUDA terletak pada kompatibilitas perangkat keras, ekosistem perangkat lunak, dan dukungan komunitas.

### Referensi
- https://www.reddit.com/r/LocalLLaMA/comments/1crwkia/why_people_buying_macs_instead_of_cuda_machines/?rdt=46933
- https://pytorch.org/docs/stable/notes/mps.html

### Key Quotes from Reddit Discussion:
- "Unified memory allows them to run bigger models in a portable way..." (LumbarJam)
- "To get 192gb of nvidia vram it's gonna be wildly expensive..." (Enough-Meringue4745)
- "Unified Memory is the draw here..." (waitmarks)
- "On Apple silicon maps, the GPU is capable of accessing way more RAM..." (ervwalter)